# Exercice Chap 3
### CRUD operations
#### Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

In [ ]:
import pymongo

# open connection at port 27017 https://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers
client = pymongo.MongoClient('localhost', 27017)
# create db tutorial
mydb = client["Todo"]
# create collection example
collection = mydb["CRUD_exercise"]

#### 1: Take the dict created in the exercise 3 Chapter II and save it in the collection "CRUD_exercise".

In [ ]:
# random dict

paper = {"authors" : ["Yann LeCun, Yoshua Bengio, Geoffrey Hinton"],
         "title" : "Deep Learning",
         "affiliations" : ["New York University","Université de Montréal,","University of Toronto"]}

collection.insert_one(paper)

#### 2: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

In [ ]:
#Creating

for i in tqdm.tqdm(range(3)):
    post = {"x":1}
    collection.insert_one(post)

# Deleting

collection.delete_one({'x': 1})

#### The oldest one is deleted

In [ ]:
# Updating

collection.update_many({'x': 1}, {'$inc': {'x': 3}})

#### 3: Insert the dict created in the exercise 6 Chapter II in the example collection.

In [ ]:
print(dict)

mydb2 = client['tutorial']
collection2 = mydb2["example"]

collection2.insert_one(dict)

#### 4: Get documents where authors key exist in the collection "CRUD_exercise".

In [ ]:
docs = collection.find({"authors" : {"$exists": True}})

for doc in docs:
    print(doc)

#### 5: Change the documents where x = 4 to x = 1.

In [ ]:
collection.update_many({'x': 4}, {'$inc': {'x': - 3}})

#### 6: Find documents where author is not_mike and set author as real_mike.

In [ ]:
collection.update_many({"author": "not_mike"},
                      {'$set': {
                          'author': "real_mike"
                          }
                      }, upsert=True)

If we're talking about the collection CRUD_exercise, then we already know there is no document where author = not_mike. However, using the True statement in upsert allows us to add the modified document in the event where not_mike doesn't exist.

#### 7: Delete documents where author is real_mike.

In [ ]:
collection.delete_many({'author': "real_mike"})

### Managing DB
#### 8: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

In [ ]:
import pymongo
import tqdm

client = pymongo.MongoClient('localhost', 27017)
mydb = client["tutorial"]
collection = mydb["CRUD_exercise_benchmark"]

list_of_insertion = []
for i in tqdm.tqdm(range(0,500000,2)):
    post = {"values":i}
    list_of_insertion.append(post)

        
collection.insert_many(list_of_insertion)


In [ ]:
unexpected_reviews = collection.find({"review": {"$regex": ".*unexpected.*"}})

# Count the number of such documents
unexpected_count = unexpected_reviews.count()

print(f"Number of reviews containing 'unexpected': {unexpected_count}")

In [ ]:
import time

start_time = time.time()
result = collection.find({"values":100000}) 
time.time() - start_time

In [ ]:
collection.create_index([ ("values",1) ])

In [ ]:
start_time = time.time()
result = collection.find({"values":100000}) 
time.time() - start_time

I don't see much difference. Maybe the difference will appear in larger collections

#### 9: create a random collection in a random db and put the new collection in the tutorial DB

In [4]:
client = pymongo.MongoClient('localhost',27017)

mydb_old = client["randomdb"]
collection_old = mydb_old["randomcoll"]

new_posts = [{"author": "Augustin",
              "title":"This is a paper from another DB",
               "text": "Another post!",
               "affiliation": "University of Strasbourg"}]

collection_old.insert_many(new_posts)

NameError: name 'pymongo' is not defined

In [ ]:
mydb = client["tutorial"] 

docs = collection_old.find()
for doc in docs:
    mydb[collection_old.name].insert_one(doc)

#### 10: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

An inner join returns only the matching values between two databses, while an outer join returns all rows from one or both of the dbs being joined, along with any matching rows from the other table.

### Real world problems
#### 11: Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

#### 12: Do the same as exercise 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

#### 13: Download a random image and store it in a collection.

In [ ]:
from PIL import Image
from matplotlib import pyplot
import numpy as np

image = Image.open("C:/Users/user/OneDrive/Pictures/Posters/مقهى ابو الياس.jpg")
pyplot.imshow(image)
pyplot.show()

In [ ]:
import numpy as np
from bson.binary import Binary
import pickle
import pymongo

data = np.asarray(image)
post = {}

post['image'] = Binary( pickle.dumps( data, protocol=2) ) 

client = pymongo.MongoClient('localhost', 27017)
mydb = client["tutorial"]
collection = mydb["image"]
collection.insert_one(post)

#### 14: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

In [ ]:
import pandas as pd

data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}

row_names = ['X', 'Y', 'Z']

df = pd.DataFrame(data, index=row_names)
print(df)

data_dict = {
    'row_names': df.index.tolist(),
    'col_names': df.columns.tolist(),
    'values': df.values.tolist()
}

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["pandas"]  # Replace "your_collection_name" with your actual collection name

# Insert the dictionary into MongoDB
collection.insert_one(data_dict)

#### 15: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

In [ ]:
df = pd.read_csv('C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/movie_review.tsv', sep='\t')
print(df)

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["movie_reviews"]  # Collection to store movie reviews

# Insert each row as a document into MongoDB
data = df.to_dict(orient='records')
collection.insert_many(data)


In [ ]:
unexpected_reviews = collection.find({"review": {"$regex": ".*unexpected.*"}})

# Count the number of such documents
unexpected_count = unexpected_reviews.count()

print(f"Number of reviews containing 'unexpected': {unexpected_count}")

#### 16: Download a sound sample. Try to store it in MongoDB

In [ ]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["sound_samples"]  # Collection to store sound samples

# Read the sound sample file
with open("C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/734512__kmita__manawatu-river.m4a", "rb") as f:
    sound_data = f.read()

# Create a document to store the sound sample
doc = {
    "filename": "sound_sample.wav",  # Name of the sound file
    "data": sound_data  # Binary data of the sound file
}

# Insert the document into MongoDB
collection.insert_one(doc)


#### 17: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ?

In [ ]:
import pymongo
import tqdm
import random

client = pymongo.MongoClient('localhost',27017)
mydb = client["exercise"]
collection = mydb["years"]

list = []
for i in tqdm.tqdm(range(300)):
    y = {"year": random.randint(2000, 2020)}
    list.append(y)
    

    
collection.insert_many(list)

In [ ]:
for doc in docs:
    print(doc)

In [ ]:
import time


start_time = time.time()
docs = collection.find({"year" : 2000})
for doc in docs:
    print(doc)
time.time() - start_time


In [ ]:
collection.create_index([ ("year",1) ])

start_time = time.time()
docss = collection.find({"year" : 2000})
for doc in docss:
    print(doc)
time.time() - start_time


Yes the indexing helps. The operation wenr from 0.00199 seconds to 0.00099

### Real World application 1

In [ ]:
import pymongo
import zipfile
import json

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["pubmed_articles"]  # Collection to store PubMed articles

with open("C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/pubmed_cleaned.json", "r") as file:
    data = json.load(file)
    for doc in data:
        doc.pop("_id", None)  # Remove the _id field
    collection.insert_many(data)

#### 1: Create an index, explain your choice of key.

In [ ]:
collection.create_index([ ("pmid",1) ])

The indexed field must be made of unique values (or at least its better taht it be made of unique values.

#### 2: Delete every paper that was published prior 2019

In [ ]:
print(collection.count_documents({'date_medline': {'$lt': 'year 2019, month 1, day 1, hour 1, minute 0'}}))

print(collection.count_documents({'date_accepted': {'$lt': 'year 2019, month 1, day 1, hour 1, minute 0'}}))


Which key represents the publishing date? If its date_medline, then there are no articles published before 2019, if it's date, then there are 3 articles. For simplification, I will use date as the publishing date.

In [ ]:
collection.delete_many({'date': {'$lt': 'year 2019, month 1, day 1, hour 1, minute 0'}})

#### 3: How many paper have a single author ? Two authors ?

In [ ]:
# Define the regular expression pattern to match "name" followed by a word boundary
pattern = r"\bname\b"

# Count the occurrences of the pattern in the authors field
one_author = collection.find({"authors": {"$regex": pattern, "$options": "m"}})

# Filter documents with exactly one occurrence of the pattern
articles_with_one = 0
articles_with_two = 0
for one in one_author:
    if one["authors"].count("name") == 1:
        articles_with_one +=1
    elif one["authors"].count("name") == 2:
        articles_with_two += 1

print("papers with one author:", articles_with_one)
print("papers with two authors:", articles_with_two) 




#### 4: What's the last paper inserted in the db ?

In [ ]:
last_paper = collection.find_one(sort=[('_id', -1)])

# Afficher les détails du dernier document
print("Dernier document inséré :")
print("Titre :", last_paper['title'])
print("Auteurs :", last_paper['authors'])
print("Date :", last_paper['date'])

#### 5: Find articles with null meshwords.

In [ ]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["pubmed_articles"]  # Collection to store PubMed articles

query = {"meshwords": {"$in": [None, ""]}}
articles_with_null_meshwords = collection.find(query)

# Affichage des résultats de la recherche
print("Articles with null meshwords:\n")
for article in articles_with_null_meshwords:
    print(article['title'])
    print(article["meshwords"])

#### 6: Choose a keyword you are interested in (machine learning, computer vision,...). Find the number of articles with the choosen keyword in their meshwords, abstract or title.

In [ ]:
from pymongo import MongoClient

keyword = "machine learning"

# Compter le nombre d'articles contenant le mot-clé dans les meshwords, le titre ou l'abstract
nombre_articles = collection.count_documents({
    "$or": [
        {"meshwords": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"abstract": {"$regex": f".*{keyword}.*", "$options": "i"}},
        {"title": {"$regex": f".*{keyword}.*", "$options": "i"}}
    ]
})

print(f"Nombre d'articles avec le mot-clé '{keyword}': {nombre_articles}")

#### 7: What's the number of articles that have atleast one affiliation AND meshwords.

In [ ]:
count = collection.count_documents({
    "$and": [
        {"affiliations": {"$exists": True}},
        {"meshwords": {"$exists": True}}
    ]
})

print(f"Nombre d'articles avec au moins une affiliation et des meshwords: {count}")

#### 8: How many articles have a publishing date after 2020 ?

In [ ]:
import pymongo

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["pubmed_articles"]  # Collection to store PubMed articles

print(collection.count_documents({'date': {'$gt': 'year 2020'}}))

#### 9: Find articles where there's atleast one affiliation from a choosen country (you decide which one).

In [ ]:
article_count = collection.count_documents({
    "authors": {"$regex": "France"}})
print("Number of articles with at least one affiliation from France:", article_count)
article_country= collection.find({"authors":{"$regex": "France"}})
for article in article_country:
    print("ID:", article["_id"],"Title :", article["title"])

#### 10: Check for any duplicates. (hint: look at the doi or the pmid)

In [ ]:
# Vérifier s'il y a des doublons

distinct_pmids = collection.distinct("pmid")

# Vérifier s'il y a des doublons
duplicates = len(distinct_pmids) - collection.count_documents({})

print("Number of duplicates:",duplicates)

#### 11: Remove every articles where the abstract starts with an "R".

In [ ]:
docs = collection.find({"abstract": {"$regex": "^\\s*\"R"}})
for doc in docs:
    print("title :", doc["title"])
    print("abstract :", doc["abstract"])



In [ ]:
print("init count: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))

In [ ]:
collection.delete_many({"abstract": {"$regex": "^\\s*\"R"}})
print("after delete many: ",collection.count_documents({"abstract": {"$regex": "^\\s*\"R"}}))

#### 12: Return the list of papers (pmid) where there's atleast one affiliation per author

In [ ]:
#Retourne la liste des articles (pmid) où chaque auteur a au moins une affiliation
articles = collection.find(
    {"authors": {"$exists": True}}
)
liste_articles = [article["pmid"] for article in articles if len(article.get("authors", [])) > 0]
print(liste_articles[0:10])

#### 13: Create 500 random samples of the dataset, compute a statistics that you are interested in and check how it behaves through the different samples

In [ ]:

num_samples = 500

#Définir la statistique d'intérêt
statistics = []

#Échantillonner et calculer la statistique pour chaque échantillon
for i in range(num_samples):
    #Échantillonner des documents aléatoirement à partir de la collection
    sample = collection.aggregate([
        {"$sample": {"size": 100}}  #Ajuster la taille de l'échantillon au besoin
    ])

    # Calculer la statistique d'intérêt pour l'échantillon
    # Ici, nous calculons la taille d'équipe moyenne
    team_sizes = [doc.get("team_size", 0) for doc in sample]
    average_team_size = sum(team_sizes) / len(team_sizes)
    
    # Ajouter la statistique calculée à la liste
    statistics.append(average_team_size)

#Afficher les statistiques
print(statistics)

### Real World application 2

In [ ]:
import pymongo
import zipfile
import json
from bson import ObjectId
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["exercise"]  # Replace "your_database_name" with your actual database name
collection = db["authors"] 

with open("C:/Users/user/OneDrive/Documents/M1-APE-DS2E/Infrastructure_de_donnes/NoSQL/authors.json", "r",encoding="utf-8") as file:
    data = json.load(file)
for doc in data:
    if '_id' in doc and '$oid' in doc['_id']:
        doc['_id'] = ObjectId(doc['_id']['$oid'])

collection.insert_many(data)

#### 1: Create an index, explain your choice of key.

In [ ]:
collection.create_index([ ("AND_ID",1) ])

AND_ID represents a unique disambiguated identifier for the author, so it makes a useful index.

#### 2: What is the average length of "pmid_list"

In [ ]:
total_length = 0
total_docs = len(data)

for doc in data:
    pmid_list_length = len(doc.get("pmid_list", []))
    total_length += pmid_list_length

average_length = total_length / total_docs
print("Average length of 'pmid_list':", int(average_length))


#### 3: How many distinct affiliations are there ?

In [ ]:
distinct_affiliations = set()

for doc in data:
    affiliations = doc.get("oa04_affiliations", [])
    for affiliation_info in affiliations:
        affiliation = affiliation_info.get("Affiliation")
        if affiliation:
            distinct_affiliations.add(affiliation)

distinct_count = len(distinct_affiliations)
print("Number of distinct affiliations:", distinct_count)

#### 4: Find authors with atleast one "COM" AffiliationType

In [ ]:
authors_with_com_affiliation = []

for doc in data:
    affiliations = doc.get("oa04_affiliations", [])
    for affiliation_info in affiliations:
        affiliation_type = affiliation_info.get("AffiliationType")
        if affiliation_type == "COM":
            authors_with_com_affiliation.append(doc["AND_ID"])
            break  # Exit inner loop once we find one COM affiliation for this author

# Remove duplicate author names (if any)
authors_with_com_affiliation = list(set(authors_with_com_affiliation))

print("Authors with at least one 'COM' AffiliationType:",len(authors_with_com_affiliation))

print("\nAuthors' AND_ID:\n\n",authors_with_com_affiliation)

#### 5: How many authors switched the AffiliationType ?

In [ ]:
authors_switched_affiliation = []

for doc in data:
    affiliation_types = []
    affiliations = doc.get("oa04_affiliations", [])
    for affiliation_info in affiliations:
        affiliation_type = affiliation_info.get("AffiliationType", "")
        affiliation_types.append(affiliation_type)
    if len(affiliation_types) > 1:
        authors_switched_affiliation.append(doc.get("AND_ID"))

num_authors_switched_affiliation = len(authors_switched_affiliation)

print("Number of authors who switched AffiliationType:", num_authors_switched_affiliation)
print("\nAND_ID of authors who switched AffiliationType:\n\n",authors_switched_affiliation)

#### 6: Find affiliation with the word "China"

In [ ]:
num_affiliations_with_china = 0
affiliations_in_china = []

for doc in data:
    affiliations = doc.get("oa04_affiliations", [])
    for affiliation_info in affiliations:
        country = affiliation_info.get("Country", "")
        if "China" in country:
            num_affiliations_with_china += 1
            affiliation = affiliation_info.get("Affiliation", "")
            affiliations_in_china.append(affiliation)

# Print the total number of affiliations with China
print("Total number of affiliations with China:", num_affiliations_with_china)

# Print the list of affiliations in China
print("\nList of affiliations in China:\n\n")
for affiliation in affiliations_in_china:
    print('-',affiliation)

#### 7: Get the pmids of papers published in 2019

In [ ]:
num_papers_2019 = 0
pmid_list = []

for doc in data:
    more_info = doc.get("more_info", [])
    for info in more_info:
        pub_year = info.get("PubYear", "")
        pmid = info.get("PMID", "")
        if pub_year == 2019:
            num_papers_2019 += 1
            pmid_list.append(pmid)

print("Number of papers published in 2019:", num_papers_2019)
print("\nList of PMIDs:\n\n",pmid_list)

#### 8: Count the number of doc with "oa06_researcher_education" OR "oa04_affiliations" key and with the "oa06_researcher_education" AND "oa04_affiliations" .

In [ ]:
count_with_any = 0
count_with_both = 0

for doc in data:
    # Check if "oa06_researcher_education" key exists
    if "oa06_researcher_education" or "oa04_affiliations" in doc:
        count_with_any += 1
        
    # Check if both keys exist
    if "oa06_researcher_education" in doc and "oa04_affiliations" in doc:
        count_with_both += 1

print("Number of documents with any key:", count_with_any)
print("Number of documents with both keys:", count_with_both)


#### 9: What's the average "BeginYear" of "oa06_researcher_education".

In [ ]:
total_begin_years = []
num_papers_with_oa06 = 0

for doc in data:
    if "oa06_researcher_education" in doc:
        for entry in doc["oa06_researcher_education"]:
            begin_year = entry.get("BeginYear")
            if begin_year is not None and begin_year != ' ':
                total_begin_years.append(begin_year)  # Convert to integer
                num_papers_with_oa06 += 1

total_begin_years_int = [int(year) for year in total_begin_years]

# Calculate the sum of the integers
average = sum(total_begin_years_int)/num_papers_with_oa06

print("Average 'BeginYear' of  'oa06_researcher_education'", int(average))


#### 10: Count the distinct country of "oa06_researcher_education"

In [ ]:
distinct_countries = collection.distinct("oa06_researcher_education.Country")

num_distinct_countries = len(distinct_countries)

print("Number of distinct countries in oa06_researcher_education:", num_distinct_countries)
print("Distinct countries:")
for country in distinct_countries:
    print(country)


#### 11: Does the length of pmid_list and more_info always match ?

In [ ]:
total_documents = collection.count_documents({})

# Counters for documents where lengths match and don't match
match_counter = 0
not_match_counter = 0

# Iterate over each document in the collection
for doc in collection.find():
    pmid_list_length = len(doc.get("pmid_list", []))  # Get length of pmid_list, default to empty list if not present
    more_info_length = len(doc.get("more_info", []))  # Get length of more_info, default to empty list if not present
    
    # Check if the lengths match
    if pmid_list_length == more_info_length:
        match_counter += 1
    else:
        not_match_counter += 1

# Calculate percentages
match_percentage = (match_counter / total_documents) * 100
not_match_percentage = (not_match_counter / total_documents) * 100

# Print percentages
print("Percentage of documents where lengths match:", match_percentage)
print("Percentage of documents where lengths don't match:", not_match_percentage)


#### 12: Does the length of pmid_list and "oa04_affiliations" always match ?

In [ ]:
total_documents = collection.count_documents({})

# Counters for documents where lengths match and don't match
match_counter = 0
not_match_counter = 0

# Iterate over each document in the collection
for doc in collection.find():
    pmid_list_length = len(doc.get("pmid_list", []))  # Get length of pmid_list, default to empty list if not present
    oa04_affiliations_length = len(doc.get("oa04_affiliations", []))  # Get length of more_info, default to empty list if not present
    
    # Check if the lengths match
    if pmid_list_length == oa04_affiliations_length:
        match_counter += 1
    else:
        not_match_counter += 1

# Calculate percentages
match_percentage = (match_counter / total_documents) * 100
not_match_percentage = (not_match_counter / total_documents) * 100

# Print percentages
print("Percentage of documents where lengths match:", match_percentage)
print("Percentage of documents where lengths don't match:", not_match_percentage)
